In [1]:
# coding: utf-8
from collections import deque
from ERDModel import RL_GRU2
import sys
import os
sys.path.append(os.path.join(os.path.dirname(os.path.curdir), '../'))
from dataUtils import *
from logger import MyLogger
os.chdir("/home/hadoop/ERD")

W0924 21:49:59.171059 139981624010560 deprecation_wrapper.py:119] From /home/hadoop/ERD/baslines/ERDProj/ERDModel.py:3: The name tf.losses.Reduction is deprecated. Please use tf.compat.v1.losses.Reduction instead.

Using TensorFlow backend.


load glove finished


In [2]:
tf.app.flags.DEFINE_string('f', '', 'kernel')

In [3]:
tf.logging.set_verbosity(tf.logging.ERROR)

logger = MyLogger("ERDMain")

In [4]:
def df_train(sess, summary_writer, mm, t_acc, t_steps, new_data_len=[]):
    sum_loss = 0.0
    sum_acc = 0.0
    ret_acc = 0.0
    init_states = np.zeros([FLAGS.batch_size, FLAGS.hidden_dim], dtype=np.float32)

    for i in range(t_steps):
        if len(new_data_len) > 0:
            x, x_len, y = get_df_batch(i, new_data_len)
        else:
            x, x_len, y = get_df_batch(i)
        feed_dic = {mm.input_x: x, mm.x_len: x_len, mm.input_y: y, mm.init_states: init_states, mm.dropout_keep_prob: 0.5}
        _, step, loss, acc = sess.run([df_train_op, df_global_step, mm.loss, mm.accuracy], feed_dic)
        summary = tf.Summary(value=[
                    tf.Summary.Value(tag="df_train_loss", simple_value=loss),
                    tf.Summary.Value(tag="df_train_accuracy", simple_value=acc),
                ])
        summary_writer.add_summary(summary, step)

        sum_loss += loss
        sum_acc += acc

        if i % 10 == 9:
            sum_loss = sum_loss / 10
            sum_acc = sum_acc / 10
            ret_acc = sum_acc
            print(get_curtime() + " Step: " + str(step) + " Training loss: " + str(sum_loss) + " accuracy: " + str(sum_acc))
            logger.info(get_curtime() + " Step: " + str(step) + " Training loss: " + str(sum_loss) + " accuracy: " + str(sum_acc))

            if sum_acc > t_acc:
                break
            sum_acc = 0.0
            sum_loss = 0.0

    print(get_curtime() + " Train df Model End.")
    logger.info(get_curtime() + " Train df Model End.")
    return ret_acc

In [5]:

def rl_train(sess, mm, t_rw, t_steps):
    ids = np.array(range(FLAGS.batch_size), dtype=np.int32)
    seq_states = np.zeros([FLAGS.batch_size], dtype=np.int32)
    isStop = np.zeros([FLAGS.batch_size], dtype=np.int32)
    max_id = FLAGS.batch_size
    init_states = np.zeros([FLAGS.batch_size, FLAGS.hidden_dim], dtype=np.float32)
    feed_dic = {mm.init_states: init_states}
    state = sess.run(mm.df_state, feed_dic)
    D = deque()
    ssq = []
    print("in RL the begining")
    logger.info("in RL the begining")
    # get_new_len(sess, mm)
    data_ID = get_data_ID()
    if len(data_ID) % FLAGS.batch_size == 0: # the total number of events
        flags = int(len(data_ID) / FLAGS.batch_size)
    else:
        flags = int(len(data_ID) / FLAGS.batch_size) + 1
    for i in range(flags):
        x, x_len, y = get_df_batch(i)
        feed_dic = {mm.input_x: x, mm.x_len: x_len, mm.input_y: y, mm.init_states:init_states, mm.dropout_keep_prob: 1.0}
        t_ssq = sess.run(mm.out_seq, feed_dic)# t_ssq = [batchsize, max_seq, scores]
        if len(ssq) > 0:
            ssq = np.append(ssq, t_ssq, axis=0)
        else:
            ssq = t_ssq

    print(get_curtime() + " Now Start RL training ...")
    logger.info(get_curtime() + " Now Start RL training ...")
    counter = 0
    sum_rw = 0.0 # sum of rewards

    data_len = get_data_len()
    while True:
        if counter > FLAGS.OBSERVE:
            sum_rw += np.mean(rw)
            if counter % 200 == 0:
                sum_rw = sum_rw / 2000
                print(get_curtime() + " Step: " + str(step) + " REWARD IS " + str(sum_rw))
                logger.info(get_curtime() + " Step: " + str(step) + " REWARD IS " + str(sum_rw))
                if sum_rw > t_rw:
                    print("Retch The Target Reward")
                    logger.info("Retch The Target Reward")
                    break
                if counter > t_steps:
                    print("Retch The Target Steps")
                    logger.info("Retch The Target Steps")
                    break
                sum_rw = 0.0
            s_state, s_x, s_isStop, s_rw = get_RL_Train_batch(D)
            feed_dic = {mm.rl_state: s_state, mm.rl_input: s_x, mm.action: s_isStop, mm.reward:s_rw, mm.dropout_keep_prob: 0.5}
            _, step = sess.run([rl_train_op, rl_global_step], feed_dic)

        x, y, ids, seq_states, max_id = get_rl_batch(ids, seq_states, isStop, max_id, 0, 3150)
        batch_dic = {mm.rl_state: state, mm.rl_input: x, mm.dropout_keep_prob: 1.0}
        isStop, mss, mNewState = sess.run([mm.isStop, mm.stopScore, mm.rl_new_state], batch_dic)

        for j in range(FLAGS.batch_size):
            if random.random() < FLAGS.random_rate:
                isStop[j] = np.argmax(np.random.rand(2))
            if seq_states[j] == data_len[ids[j]]:
                isStop[j] = 1

        # eval
        rw = get_reward(isStop, mss, ssq, ids, seq_states)

        for j in range(FLAGS.batch_size):
            D.append((state[j], x[j], isStop[j], rw[j]))
            if len(D) > FLAGS.max_memory:
                D.popleft()

        state = mNewState
        for j in range(FLAGS.batch_size):
            if isStop[j] == 1:
                # init_states = np.zeros([FLAGS.batch_size, FLAGS.hidden_dim], dtype=np.float32)
                # feed_dic = {mm.init_states: init_states}
                # state[j] = sess.run(mm.df_state, feed_dic)
                state[j] = np.zeros([FLAGS.hidden_dim], dtype=np.float32)
        counter += 1

In [6]:
eval_flag

0

In [7]:
def eval(sess, mm):
#     start_ef = int(eval_flag / FLAGS.batch_size)
#     end_ef = int(len(data_ID) / FLAGS.batch_size) + 1
    data_ID = get_data_ID()
    batchs = int( len(data_ID)/FLAGS.batch_size )
    init_states = np.zeros([FLAGS.batch_size, FLAGS.hidden_dim], dtype=np.float32)

    counter = 0
    sum_acc = 0.0

    for i in range(batchs):
        x, x_len, y = get_df_batch(i)
        feed_dic = {mm.input_x: x, mm.x_len: x_len, mm.input_y: y, mm.init_states: init_states, mm.dropout_keep_prob: 1.0}
        loss, acc = sess.run([mm.loss, mm.accuracy], feed_dic)
        print("loss:",loss, "| acc:", acc)
        counter += 1
        sum_acc += acc

    print(sum_acc / counter)

In [9]:
print(get_curtime() + " Loading data ...")
logger.info(get_curtime() + " Loading data ...")
# load_data(FLAGS.data_file_path)
load_data_fast()
print(get_curtime() + " Data loaded.")
logger.info(get_curtime() + " Data loaded.")
gpu_config = tf.ConfigProto(allow_soft_placement=True, log_device_placement=True)

gpu_config.gpu_options.allow_growth=True
# (self, input_dim, hidden_dim, max_seq_len, max_word_len, class_num, action_num):
print(FLAGS.embedding_dim, FLAGS.hidden_dim, FLAGS.max_seq_len, FLAGS.max_sent_len, FLAGS.class_num, FLAGS.action_num)
logger.info((FLAGS.embedding_dim, FLAGS.hidden_dim, FLAGS.max_seq_len, FLAGS.max_sent_len, FLAGS.class_num, FLAGS.action_num))

I0924 22:12:15.169942 139981624010560 logger.py:24] 2019-09-24 22:12:15 Loading data ...
I0924 22:12:15.370373 139981624010560 logger.py:24] 2019-09-24 22:12:15 Data loaded.


2019-09-24 22:12:15 Loading data ...
max_sent: 187 ,  max_seq_len: 101
5802 data loaded
2019-09-24 22:12:15 Data loaded.


I0924 22:12:15.371721 139981624010560 logger.py:24] (300, 200, 101, 187, 2, 2)


300 200 101 187 2 2


In [10]:
sess = tf.Session(config=gpu_config)
with  sess.as_default():
    with tf.device('/GPU:0'):
        mm = RL_GRU2(FLAGS.embedding_dim, FLAGS.hidden_dim, FLAGS.max_seq_len,
                     FLAGS.max_sent_len, FLAGS.class_num, FLAGS.action_num, FLAGS.sent_num)

        # df model
        df_global_step = tf.Variable(0, name="global_step", trainable=False)
        df_train_op = tf.train.AdagradOptimizer(0.05).minimize(mm.loss, df_global_step)

        # rl model
        rl_global_step = tf.Variable(0, name="global_step", trainable=False)
        rl_train_op = tf.train.AdamOptimizer(0.001).minimize(mm.rl_cost, rl_global_step)

        saver = tf.train.Saver(tf.global_variables(), max_to_keep=4)
        sess.run(tf.global_variables_initializer())

In [10]:
d_IDs = get_data_ID()

In [11]:
len(d_IDs)

4584

### randomly initialized model

In [12]:
with tf.device('/GPU:0'):
    eval(sess, mm)

loss: 0.9761976 | acc: 0.64
loss: 1.0305308 | acc: 0.62
loss: 0.93481076 | acc: 0.66
loss: 0.79329556 | acc: 0.74
loss: 0.9009328 | acc: 0.66
loss: 0.9347645 | acc: 0.64
loss: 1.0222572 | acc: 0.6
loss: 0.8108392 | acc: 0.7
loss: 1.1172113 | acc: 0.56
loss: 0.99594104 | acc: 0.6
loss: 1.008531 | acc: 0.6
loss: 0.8904818 | acc: 0.66
loss: 0.827948 | acc: 0.7
loss: 0.9455811 | acc: 0.64
loss: 0.9736242 | acc: 0.62
loss: 0.80959874 | acc: 0.72
loss: 1.0085737 | acc: 0.62
loss: 0.89355296 | acc: 0.68
loss: 0.82728916 | acc: 0.7
loss: 0.98621595 | acc: 0.62
loss: 1.0144815 | acc: 0.62
loss: 1.062881 | acc: 0.58
loss: 1.1081119 | acc: 0.56
loss: 0.7471452 | acc: 0.76
loss: 0.8067429 | acc: 0.72
loss: 0.9624578 | acc: 0.64
loss: 0.7500696 | acc: 0.74
loss: 1.0621426 | acc: 0.6
loss: 0.8379242 | acc: 0.7
loss: 0.95734334 | acc: 0.62
loss: 0.8648909 | acc: 0.68
loss: 1.1910346 | acc: 0.52
loss: 0.85524356 | acc: 0.7
loss: 0.7599956 | acc: 0.76
loss: 0.76777035 | acc: 0.74
loss: 0.9485195 | acc:

In [13]:
ckpt_dir = "df_saved_erd"
checkpoint = tf.train.get_checkpoint_state(ckpt_dir)
if checkpoint and checkpoint.model_checkpoint_path:
    print("--------------Debug1------------------")
    saver.restore(sess, checkpoint.model_checkpoint_path)
    print(checkpoint.model_checkpoint_path+" is restored.")
    logger.info(checkpoint.model_checkpoint_path+" is restored.")

I0924 22:15:10.602153 139981624010560 logger.py:24] df_saved_erd/model14 is restored.


--------------Debug1------------------
df_saved_erd/model14 is restored.


## test on the trainning data set

In [12]:
with tf.device('/GPU:0'):
    eval(sess, mm)

loss: 0.68944013 | acc: 0.64
loss: 0.71958584 | acc: 0.62
loss: 0.67800266 | acc: 0.66
loss: 0.5913662 | acc: 0.74
loss: 0.67395717 | acc: 0.66
loss: 0.7013041 | acc: 0.64
loss: 0.7399647 | acc: 0.6
loss: 0.63522685 | acc: 0.7
loss: 0.7940782 | acc: 0.56
loss: 0.7490343 | acc: 0.6
loss: 0.74350744 | acc: 0.6
loss: 0.6772332 | acc: 0.66
loss: 0.63835645 | acc: 0.7
loss: 0.69375825 | acc: 0.64
loss: 0.7240085 | acc: 0.62
loss: 0.6174975 | acc: 0.72
loss: 0.72881204 | acc: 0.62
loss: 0.6437065 | acc: 0.68
loss: 0.6357735 | acc: 0.7
loss: 0.7205063 | acc: 0.62
loss: 0.71788687 | acc: 0.62
loss: 0.76473695 | acc: 0.58
loss: 0.79534405 | acc: 0.56
loss: 0.5719873 | acc: 0.76
loss: 0.6108683 | acc: 0.72
loss: 0.709652 | acc: 0.64
loss: 0.5881363 | acc: 0.74
loss: 0.74533266 | acc: 0.6
loss: 0.6309216 | acc: 0.7
loss: 0.71896046 | acc: 0.62
loss: 0.6508034 | acc: 0.68
loss: 0.82587147 | acc: 0.52
loss: 0.63478255 | acc: 0.7
loss: 0.5728295 | acc: 0.76
loss: 0.58607113 | acc: 0.74
loss: 0.69536

KeyboardInterrupt: 

## Test on another dataset

In [14]:
with tf.device('/GPU:0'):
    eval(sess, mm)

loss: 1.6896323 | acc: 0.54
loss: 2.0606124 | acc: 0.44
loss: 1.4072715 | acc: 0.62
loss: 2.3717637 | acc: 0.34
loss: 1.5512682 | acc: 0.58
loss: 1.975043 | acc: 0.46
loss: 2.041068 | acc: 0.44
loss: 1.9526526 | acc: 0.46
loss: 1.7462956 | acc: 0.52
loss: 1.8309494 | acc: 0.5
0.4900000005960464


In [ ]:
with  sess.as_default():
    with tf.device('/GPU:0'):
        mm = RL_GRU2(FLAGS.embedding_dim, FLAGS.hidden_dim, FLAGS.max_seq_len,
                     FLAGS.max_sent_len, FLAGS.class_num, FLAGS.action_num, FLAGS.sent_num)

        # df model
        df_global_step = tf.Variable(0, name="global_step", trainable=False)
        df_train_op = tf.train.AdagradOptimizer(0.05).minimize(mm.loss, df_global_step)

        # rl model
        rl_global_step = tf.Variable(0, name="global_step", trainable=False)
        rl_train_op = tf.train.AdamOptimizer(0.001).minimize(mm.rl_cost, rl_global_step)

        saver = tf.train.Saver(tf.global_variables(), max_to_keep=4)
        sess.run(tf.global_variables_initializer())

summary_writer = tf.summary.FileWriter("./reports/", graph=sess.graph)
df_train(sess, summary_writer, mm, 0.90, 20000)

for i in range(20):
    rl_train(sess, mm, 0.5, 50000)
    saver.save(sess, "rl_saved_erd/model"+str(i))
    print("rl_model "+str(i)+" saved")
    logger.info("rl_model "+str(i)+" saved")
    new_len = get_new_len(sess, mm)
    acc = df_train(sess, summary_writer, mm, 0.9, 1000, new_len)
    saver.save(sess, "df_saved_erd/model"+str(i))
    print("df_model "+str(i)+" saved")
    logger.info("df_model "+str(i)+" saved")
    if acc > 0.9:
        break

print("The End of My Program")
logger.info("The End of My Program")
